In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import tqdm

# Read image
files  = sorted(glob.glob("/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019/ISIC_2019_Training_Input/*.jpg"))
print(len(files))
# for f in tqdm.tqdm(files):
#     img = cv2.imread(f, cv2.IMREAD_COLOR)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     # resize to 224x224
#     img = cv2.resize(img, (224, 224),cv2.INTER_AREA )
#     # save over original
#     cv2.imwrite(f, img)


25331


In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision.models import ResNet101_Weights, resnet101

# Load pretrained model
# model = resnet101(weights=ResNet101_Weights.IMAGENET1K_V2)
model = resnet101()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import pandas as pd

df_train = pd.read_csv("/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019/ISIC_2019_Training_GroundTruth.csv")
print(df_train.head())

          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


### Applying color constancy

In [4]:
def color_constancy(img, power=6, gamma=None):
    """
    Parameters
    ----------
    img: 3D np array
        The original image with format of (h, w, c)
    power: int
        The degree of norm, 6 is used in reference paper
    gamma: float
        The value of gamma correction, 2.2 is used in reference paper
    """
    img_dtype = img.dtype


    if gamma is not None:
        img = img.astype('uint8')
        look_up_table = np.ones((256,1), dtype='uint8') * 0
        for i in range(256): look_up_table[i][0] = 255*pow(i/255, 1/gamma)
        img = cv2.LUT(img, look_up_table)


    img = img.astype('float32')
    img_power = np.power(img, power)
    rgb_vec = np.power(np.mean(img_power, (0,1)), 1/power)
    rgb_norm = np.sqrt(np.sum(np.power(rgb_vec, 2.0)))
    rgb_vec = rgb_vec/rgb_norm
    rgb_vec = 1/(rgb_vec*np.sqrt(3))
    img = np.multiply(img, rgb_vec)
    
    return img.astype(img_dtype) 


In [5]:
from sklearn.model_selection import train_test_split
import os
def load_isic_2019(classes):
    root = "/ssd_scratch/cvit/anirudhkaushik/dataset/ISIC_2019"
    """
    Load ISIC_2019 dataset and convert it to IIRC format

    Args:
        root (string): The location of the dataset
        intask_valid_train_ratio (float): the percentage of the training set to be taken for the in-task validation set
            , a training-like validation set used for valdation during the task training (default: 0.1)
        posttask_valid_train_ratio (float): the percentage of the training set to be taken for the post-task validation
            set, a test-like validation set used for valdation after the task training (default: 0.1)

    Returns:
        Dict[str, DatasetStructType]: datasets, a dictionary with the keys corresponding to the four splits (train,
        intask_validation, posttask_validation, test), and the values being a list of the samples that belong to
        each split (with the images provided in Image.Image type) in the DatasetTypeStruct structure
    """
    raw_data_meta_df = pd.read_csv(root+'/ISIC_2019_Training_GroundTruth.csv')

    isic_data_map = {
        "MEL": "Melanoma",  
        "NV": "Melanocytic_nevus" ,
        "BCC": "Basal_cell_carcinoma",
        "AK": "Actinic_keratosis",
        "BKL": "Benign_keratosis",
        "DF": "Dermatofibroma",
        "VASC": "Vascular_lesion",
        "SCC": "Squamous_cell_carcinoma"
    }

    train_data_classwise = {class_name: [] for class_name in classes}
    test_data_classwise = {class_name: [] for class_name in classes}
    
    train_num_samples_per_class = {class_name: 0 for class_name in classes}
    test_num_samples_per_class = {class_name: 0 for class_name in classes}
    
    labels = list(raw_data_meta_df.columns[1:-1])
    class_to_idx = {isic_data_map[label]: idx for idx, label in enumerate(classes)}


    X = raw_data_meta_df.iloc[:]['image'] # only image names, not actual images
    y = raw_data_meta_df.iloc[:, 1:]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1, stratify=y)

    raw_data_train = []
    for ind  in range(len(X_train)):
        img_name = X_train.iloc[ind]
        labels = y_train.iloc[ind]
        label = labels[labels == 1].index[0]
        label_name = label
        if label not in classes:
            continue            
        image = cv2.imread(os.path.join(root, "ISIC_2019_Training_Input", img_name+".jpg"), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224), cv2.INTER_AREA) # remove later
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = color_constancy(image) 
        label = class_to_idx[isic_data_map[label]] 
        # raw_data_train.append((image, label))
        train_data_classwise[label_name].append((image, label))


    # Balance as per the class with minimum number of samples
    min_num_samples = min([len(train_data_classwise[class_name]) for class_name in classes])
    # randomly sample from other classes to make the number of samples equal to min_num_samples
    for class_name in classes:
        np.random.shuffle(train_data_classwise[class_name])
        train_data_classwise[class_name] = train_data_classwise[class_name][:min_num_samples]
        train_num_samples_per_class[class_name] = len(train_data_classwise[class_name])

    print("Train data distribution")
    for class_name in classes:
        label_name = isic_data_map[class_name]
        print("{}: {}".format(label_name, train_num_samples_per_class[class_name]))

    # insert the samples into the raw_data_train fast
    for class_name in classes:
        raw_data_train.extend(train_data_classwise[class_name])




    raw_data_test = []
    for ind  in range(len(X_test)):
        img_name = X_test.iloc[ind]
        labels = y_test.iloc[ind]
        label = labels[labels == 1].index[0]
        label_name = label
        if label not in classes:
            continue
        image = cv2.imread(os.path.join(root, "ISIC_2019_Training_Input", img_name+".jpg"), cv2.IMREAD_COLOR)
        image = cv2.resize(image, (224, 224), cv2.INTER_AREA) # remove later, inter area is for making it smaller, for making it larger use inter linear
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = color_constancy(image) 
        label = class_to_idx[isic_data_map[label]]
        # raw_data_test.append((image, label))
        test_data_classwise[label_name].append((image, label))

    # Balance as per the class with minimum number of samples
    min_num_samples = min([len(test_data_classwise[class_name]) for class_name in classes])
    # randomly sample from other classes to make the number of samples equal to min_num_samples
    for class_name in classes:
        np.random.shuffle(test_data_classwise[class_name])
        test_data_classwise[class_name] = test_data_classwise[class_name][:min_num_samples]
        test_num_samples_per_class[class_name] = len(test_data_classwise[class_name])

    # insert the samples into the raw_data_test fast
    for class_name in classes:
        raw_data_test.extend(test_data_classwise[class_name])

    print("Test data distribution")
    for class_name in classes:
        label_name = isic_data_map[class_name]
        print("{}: {}".format(label_name, test_num_samples_per_class[class_name]))

    return raw_data_train, raw_data_test

In [6]:
train_nevus, test_nevus = load_isic_2019(["MEL", "NV"])
train_bkl, test_bkl = load_isic_2019(["MEL", "BKL"])
final_train, final_test = load_isic_2019(["MEL", "NV", "BKL"])

print()

Train data distribution
Melanoma: 4070
Melanocytic_nevus: 4070
Test data distribution
Melanoma: 452
Melanocytic_nevus: 452
Train data distribution
Melanoma: 2361
Benign_keratosis: 2361
Test data distribution
Melanoma: 263
Benign_keratosis: 263
Train data distribution
Melanoma: 2361
Melanocytic_nevus: 2361
Benign_keratosis: 2361
Test data distribution
Melanoma: 263
Melanocytic_nevus: 263
Benign_keratosis: 263



In [7]:
base_model_path = "/ssd_scratch/cvit/anirudhkaushik/saved_models/"

#### Melanoma vs Nevus 
 *Note: this model was trained with dysplastic Nevus (suspected) trained without color constancy*

In [8]:
melanoma_vs_nevus_model = resnet101()
melanoma_vs_nevus_model.fc = nn.Linear(2048, 2)
melanoma_vs_nevus_model.load_state_dict(torch.load(os.path.join(base_model_path, "dysplastic_nevus_exp2.pth")))
melanoma_vs_nevus_model = torch.nn.DataParallel(model)
melanoma_vs_nevus_model = melanoma_vs_nevus_model.to(device)

#### Melanoma vs Seborrheic Keratosis
*Note: this model wil be trained from scratch*

In [9]:
melanoma_vs_benign_keratosis_model = resnet101()
melanoma_vs_benign_keratosis_model.fc = nn.Linear(2048, 2)
melanoma_vs_benign_keratosis_model = torch.nn.DataParallel(model)
melanoma_vs_benign_keratosis_model = melanoma_vs_benign_keratosis_model.to(device)

### Final model
*Note: this model will be trained from scratch on all 3 classes jointly and will be used as a baseline for melanoma identification*

In [10]:
mel_nv_bkl_model = resnet101()
mel_nv_bkl_model.fc = nn.Linear(2048, 3)
mel_nv_bkl_model = torch.nn.DataParallel(model)
mel_nv_bkl_model = mel_nv_bkl_model.to(device)

In [11]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# dataloader
class ISICDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img, label = self.data[idx]
        if self.transform:
            img = self.transform(img)
        return img, label
    
# transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225] )
])


In [12]:
# mel vs nevus dataset and dataloader
train_mel_vs_nevus_dataset = ISICDataset(train_nevus, transform=transform)
test_mel_vs_nevus_dataset = ISICDataset(test_nevus, transform=transform)

train_mel_vs_nevus_dataloader = DataLoader(train_mel_vs_nevus_dataset, batch_size=32, shuffle=True, num_workers=4)
test_mel_vs_nevus_dataloader = DataLoader(test_mel_vs_nevus_dataset, batch_size=32, shuffle=True, num_workers=4)

# mel vs bkl dataset and dataloader
train_mel_vs_bkl_dataset = ISICDataset(train_bkl, transform=transform)
test_mel_vs_bkl_dataset = ISICDataset(test_bkl, transform=transform)

train_mel_vs_bkl_dataloader = DataLoader(train_mel_vs_bkl_dataset, batch_size=32, shuffle=True, num_workers=4)
test_mel_vs_bkl_dataloader = DataLoader(test_mel_vs_bkl_dataset, batch_size=32, shuffle=True, num_workers=4)

# final dataset and dataloader
train_final_dataset = ISICDataset(final_train, transform=transform)
test_final_dataset = ISICDataset(final_test, transform=transform)

train_final_dataloader = DataLoader(train_final_dataset, batch_size=32, shuffle=True, num_workers=4)
test_final_dataloader = DataLoader(test_final_dataset, batch_size=32, shuffle=True, num_workers=4)

In [13]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

def test_model(model, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)

            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    print('\nTest set: Accuracy: {:.0f}%\n'.format(
            100. * correct / len(test_loader.dataset)))
    
    return correct / len(test_loader.dataset)

def train_model(model, train_loader, test_loader, optimizer, epochs):
    acc_list = []
    loss_list = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss/100))
                running_loss = 0.0
        
        acc = test_model(model, test_loader)
        acc_list.append(acc)
        loss_list.append(running_loss)

    return acc_list, loss_list



In [14]:
def classwise_acc(model, test_loader, classes):
    isic_data_map = {
        "MEL": "Melanoma",  
        "NV": "Melanocytic_nevus" ,
        "BCC": "Basal_cell_carcinoma",
        "AK": "Actinic_keratosis",
        "BKL": "Benign_keratosis",
        "DF": "Dermatofibroma",
        "VASC": "Vascular_lesion",
        "SCC": "Squamous_cell_carcinoma"
    }
    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}

    # again no gradients needed
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            # collect the correct predictions for each class
            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    # print accuracy for each class
    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')
    return [correct_pred[isic_data_map[classname]] / total_pred[isic_data_map[classname]] for classname in classes]


In [15]:
def summarise_experiment(model, classes, train_dataloader, test_dataloader, save=False, save_path=None):

    # train and test accuracy
    acc_list, loss_list = train_model(model,train_dataloader,test_dataloader, optimizer, 14)
    # plot accuracy and loss curves
    plt.subplot(2, 1, 1)
    plt.plot(acc_list)
    plt.title("Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")

    plt.subplot(2, 1, 2)
    plt.plot(loss_list)
    plt.title("Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")

    plt.tight_layout()
    plt.show()

    # print final test accuracy
    final_test_acc_bkl = test_model(model, test_dataloader)
    print("Final test accuracy: {:.0f}%".format(100. * final_test_acc_bkl))

    # print classwise accuracy
    classwise_score = classwise_acc(model, test_dataloader, classes)
    print("Melanoma class accuracy: {:.0f}%".format(100. * classwise_score[0]))
    print("Melanocytic Nevus class accuracy: {:.0f}%".format(100. * classwise_score[1]))

    # print mean and standard deviation of the training
    # mean and standard deviation of accuracy 
    mean = np.mean(acc_list)
    std = np.std(acc_list)
    print("Mean accuracy: {:.2f}%".format(100. * mean))
    print("Standard deviation of accuracy: {:.2f}%".format(100. * std))

    if save:
        torch.save(model.state_dict(), save_path)

#### Begin training Melanoma vs Benign keratosis model

In [16]:
summarise_experiment(melanoma_vs_benign_keratosis_model,["MEL", "BKL"] ,train_mel_vs_bkl_dataloader, test_mel_vs_bkl_dataloader, save=True, save_path=os.path.join(base_model_path, "melanoma_screening_exp1_mel_vs_bkl.pth"))

#### Train final model 
 - Trained on melanoma vs benign keratosis vs Nevus

In [ ]:
summarise_experiment(mel_nv_bkl_model, ["MEL", "NV", "BKL"], train_final_dataloader, test_final_dataloader, save=True, save_path=os.path.join(base_model_path, "melanoma_screening_exp1_mel_nv_bkl.pth"))

## Visualizations

In [ ]:
classes = {0: "Melanoma", 1: "Melanocytic Nevus"}
random_sampler = torch.utils.data.RandomSampler(test_dataset_refined_fixed, num_samples=10, replacement=False)
# plot these 10 images with their labels in a grid
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
for i, idx in enumerate(random_sampler):
    img, label = test_dataset_refined_fixed[idx]
    output = model(img.unsqueeze(0).to(device))
    pred = output.argmax(dim=1, keepdim=True)
    img = img.cpu().numpy()
    img = np.transpose(img, (1, 2, 0))
    # normalize
    img = (img - np.min(img)) / (np.max(img) - np.min(img))
    ax = axes[i//5, i%5]
    ax.imshow(img)
    ax.set_title("GT: {}\nPred: {}".format(classes[label], classes[pred.item()]))
    ax.set_xticks([])
    ax.set_yticks([])
plt.tight_layout()
plt.show()

